In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
print("ANALISI STARTUP 2020-2024")
print("=" * 60)

ANALISI STARTUP 2020-2024


In [2]:
# LEGGENDE CORRETTE (verificate dal dataset reale - registro imprese "startup.xls")
LEGENDA_PRODUZIONE = {
    'A': '0-100k €', 'B': '100k-500k €', 'C': '500k-1M €', 'D': '1M-2M €',
    'E': '2M-5M €', 'F': '5M-10M €', 'G': '10M-50M €', 'H': '50M+ €'
}

LEGENDA_ADDETTI = {
    'A': '0-4 addetti', 'B': '5-9 addetti', 'C': '10-19 addetti',
    'D': '20-49 addetti', 'E': '50-249 addetti', 'F': '250+ addetti'
}

LEGENDA_CAPITALE = {
    '1': '1 €', '2': '1-5k €', '3': '5-10k €', '4': '10-50k €',
    '5': '50-100k €', '6': '100-250k €', '7': '250-500k €',
    '8': '500k-1M €', '9': '1-2.5M €', '10': '2.5-5M €', '11': '5M+ €'
}
REQUISITI_STARTUP = {
    '1': '15% costi per R&D',
    '2': '2/3 team con laurea magistrale o 1/3 con dottorato',
    '3': 'Brevetti/software registrato'
}
LEGENDA_PREVALENZA = {
    'NO' : '[% del capitale sociale + % Amministratori] / 2 ≤ 50%',
    'Maggioritaria' : '[% del capitale sociale + % Amministratori] / 2 > 50%',
    'Forte' : '[% del capitale sociale + % Amministratori] / 2 > 66%',
    'Esclusiva' : '[% del capitale sociale + % Amministratori] / 2 = 100%'
}

print("\n (1) Legenda classe di produzione :", LEGENDA_PRODUZIONE)
print("\n (2) Legenda classe di addetti:", LEGENDA_ADDETTI)
print("\n (3) Legenda classe di capitale:", LEGENDA_CAPITALE)
print("\n (4) Nota su requisiti per startup:", REQUISITI_STARTUP)
print("\n (5) Legenda prevalenza femminile/giovanile/straniera:", LEGENDA_PREVALENZA)


 (1) Legenda classe di produzione : {'A': '0-100k €', 'B': '100k-500k €', 'C': '500k-1M €', 'D': '1M-2M €', 'E': '2M-5M €', 'F': '5M-10M €', 'G': '10M-50M €', 'H': '50M+ €'}

 (2) Legenda classe di addetti: {'A': '0-4 addetti', 'B': '5-9 addetti', 'C': '10-19 addetti', 'D': '20-49 addetti', 'E': '50-249 addetti', 'F': '250+ addetti'}

 (3) Legenda classe di capitale: {'1': '1 €', '2': '1-5k €', '3': '5-10k €', '4': '10-50k €', '5': '50-100k €', '6': '100-250k €', '7': '250-500k €', '8': '500k-1M €', '9': '1-2.5M €', '10': '2.5-5M €', '11': '5M+ €'}

 (4) Nota su requisiti per startup: {'1': '15% costi per R&D', '2': '2/3 team con laurea magistrale o 1/3 con dottorato', '3': 'Brevetti/software registrato'}

 (5) Legenda prevalenza femminile/giovanile/straniera: {'NO': '[% del capitale sociale + % Amministratori] / 2 ≤ 50%', 'Maggioritaria': '[% del capitale sociale + % Amministratori] / 2 > 50%', 'Forte': '[% del capitale sociale + % Amministratori] / 2 > 66%', 'Esclusiva': '[% del cap

In [4]:
def load_and_analyze_data():
    """Carica e analizza il dataset con gestione missing values"""
    
    # Carica dataset
    try:
        df = pd.read_excel('Copia di startup.xls', sheet_name='STARTUP_16062025')
        print("✅ Dataset caricato dal file Excel")
    except:
        print("❌ Errore nel caricamento file Excel")
        return None
    
    print(f"Dataset iniziale: {len(df):,} startup")
    
    # STEP 1: Pulizia nomi colonne
    df.columns = df.columns.str.strip()
    
    # STEP 2: Filtro 2020-2024 con gestione errori
    date_col = 'data iscrizione alla sezione delle startup'
    df[date_col] = pd.to_datetime(df[date_col], format='%d/%m/%Y', errors='coerce')
    df_filtered = df[df[date_col].dt.year.between(2020, 2024)].copy()
    
    print(f"Dopo filtro 2020-2024: {len(df_filtered):,} startup")
    
    # STEP 3: Analisi missing values PRIMA del mapping
    analyze_missing_values(df_filtered)
    
    # STEP 4: Pulizia colonne critiche
    df_clean = clean_critical_columns(df_filtered)
    
    return df_clean
df

NameError: name 'df_clean' is not defined

In [10]:
# vediamo le dimensioni del dataset, il numero di colonne e il tipo di variabili
print("Dimensioni del dataset:", df.shape)
print("Numero di colonne:", df.shape[1])
print("Tipo di variabili:", df.dtypes)

NameError: name 'df' is not defined

In [11]:
def analyze_missing_values(df):
    """Analizza i valori mancanti per colonne critiche"""
    
    print("\n ANALISI VALORI MANCANTI:")
    print("-" * 40)
    
    critical_cols = {
        'classe di produzione ultimo anno (1)': 'Produzione',
        'classe di addetti ultimo anno (2)': 'Addetti',
        'classe di capitale (3)': 'Capitale',
        '1° req.\n(6)': '1° Requisito',
        '2° req.\n(6)': '2° Requisito',
        '3° req.\n(6)': '3° Requisito',
        'impresa a vocazione sociale (5)': 'Vocazione Sociale',
        'sito internet': 'Sito Web'
    }
    
    missing_stats = []
    for col, name in critical_cols.items():
        if col in df.columns:
            missing = df[col].isna().sum() + (df[col] == '').sum()
            percentage = (missing / len(df)) * 100
            missing_stats.append({
                'Colonna': name,
                'Missing': missing,
                'Totale': len(df),
                'Percentuale': f"{percentage:.1f}%"
            })
            print(f"{name}: {missing:,}/{len(df):,} ({percentage:.1f}%)")
    
    return missing_stats

In [12]:
def clean_critical_columns(df):
    """Pulisce e standardizza le colonne critiche"""
    
    print("\n PULIZIA DATI:")
    print("-" * 30)
    
    # Normalizza valori stringa
    string_cols = ['classe di produzione ultimo anno (1)', 'classe di addetti ultimo anno (2)']
    for col in string_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().replace('nan', np.nan)
    
    # CAPITALE - PULIZIA CORRETTA (mantiene formato originale)
    if 'classe di capitale (3)' in df.columns:
        # Converti a stringa, pulisci, ma mantieni formato originale
        df['classe di capitale (3)'] = df['classe di capitale (3)'].astype(str).str.strip()
        # Sostituisci valori vuoti con NaN
        df['classe di capitale (3)'] = df['classe di capitale (3)'].replace(['', 'nan', 'None'], np.nan)
        # Per valori numerici con decimali, rimuovi .0
        def clean_capital_value(val):
            if pd.isna(val) or val == 'nan':
                return np.nan
            try:
                # Se è un numero con .0, rimuovi il .0
                if '.' in str(val) and str(val).endswith('.0'):
                    return str(int(float(val)))
                return str(val)
            except:
                return np.nan
        df['classe di capitale (3)'] = df['classe di capitale (3)'].apply(clean_capital_value)
    
    # Normalizza requisiti (SI/NO)
    req_cols = ['1° req.\n(6)', '2° req.\n(6)', '3° req.\n(6)']
    for col in req_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper().replace('NAN', np.nan)
    
    print("✅ Pulizia completata")
    return df

In [13]:
def enrich_with_legend_safe(df):
    """Arricchisce il dataset gestendo CORRETTAMENTE i missing values"""
    
    print("\n🚀 ARRICCHIMENTO CON GESTIONE MISSING VALUES:")
    print("-" * 50)
    
    # 1. PRODUZIONE - con gestione missing
    prod_col = 'classe di produzione ultimo anno (1)'
    if prod_col in df.columns:
        df['produzione_descrizione'] = df[prod_col].map(LEGENDA_PRODUZIONE)
        df['produzione_categoria'] = df[prod_col].apply(safe_categorize_production)
        print(f"✅ Produzione: {df['produzione_descrizione'].notna().sum():,} mappate")
    
    # 2. ADDETTI - con gestione missing  
    add_col = 'classe di addetti ultimo anno (2)'
    if add_col in df.columns:
        df['addetti_descrizione'] = df[add_col].map(LEGENDA_ADDETTI)
        df['addetti_categoria'] = df[add_col].apply(safe_categorize_employees)
        print(f"✅ Addetti: {df['addetti_descrizione'].notna().sum():,} mappate")
    
    # 3. CAPITALE - con gestione missing
    cap_col = 'classe di capitale (3)'
    if cap_col in df.columns:
        df['capitale_descrizione'] = df[cap_col].map(LEGENDA_CAPITALE)
        df['capitale_categoria'] = df[cap_col].apply(safe_categorize_capital)
        print(f"✅ Capitale: {df['capitale_descrizione'].notna().sum():,} mappate")
    
    # 4. ETA' AZIENDALE
    date_col = 'data iscrizione alla sezione delle startup'
    if date_col in df.columns:
        df['company_age_years'] = (pd.Timestamp.now() - df[date_col]).dt.days / 365.25
        df['company_age_years'] = df['company_age_years'].round(1)
        print(f"✅ Età calcolata per {df['company_age_years'].notna().sum():,} startup")
    
    # 5. CARATTERISTICHE INNOVATIVE
    tech_col = 'impresa ad alto valore tecnologico in ambito energetico(4)'
    social_col = 'impresa a vocazione sociale (5)'
    
    if tech_col in df.columns:
        df['is_tech_energy'] = (df[tech_col].str.upper() == 'SI')
    if social_col in df.columns:
        df['is_social_impact'] = (df[social_col].str.upper() == 'SI')
    
    df['has_website'] = df['sito internet'].notna() & (df['sito internet'].str.strip() != '')
    
    # 6. REQUISITI INNOVAZIONE - gestione sicura
    req_cols = ['1° req.\n(6)', '2° req.\n(6)', '3° req.\n(6)']
    df['requisiti_soddisfatti'] = 0
    
    for col in req_cols:
        if col in df.columns:
            df['requisiti_soddisfatti'] += (df[col] == 'SI').astype(int)
    
    df['innovation_level'] = df['requisiti_soddisfatti'].apply(classify_innovation)
    
    # 7. DATA QUALITY SCORE
    df['data_completeness_score'] = calculate_data_quality_score(df)
    df['data_quality_tier'] = df['data_completeness_score'].apply(classify_data_quality)
    
    # 8. BUSINESS METRICS - solo per dati completi
    df['startup_score'] = calculate_startup_score_safe(df)
    df['startup_tier'] = df['startup_score'].apply(classify_tier)
    
    print("✅ Arricchimento completato con gestione missing values!")
    return df

def safe_categorize_production(classe):
    """Categorizza produzione gestendo missing values"""
    if pd.isna(classe) or classe == 'nan':
        return 'Dati non disponibili'
    elif classe in ['A', 'B']:
        return 'Micro (0-500k)'  
    elif classe in ['C', 'D']:
        return 'Piccola (500k-2M)'
    elif classe in ['E', 'F']:
        return 'Media (2M-10M)'
    elif classe in ['G', 'H']:
        return 'Grande (10M+)'
    else:
        return 'Non classificata'

def safe_categorize_employees(classe):
    """Categorizza addetti gestendo missing values"""
    if pd.isna(classe) or classe == 'nan':
        return 'Dati non disponibili'
    elif classe in ['A', 'B']:
        return 'Micro (0-9)'
    elif classe in ['C', 'D']:
        return 'Piccola (10-49)'
    elif classe == 'E':
        return 'Media (50-249)'
    elif classe == 'F':
        return 'Grande (250+)'
    else:
        return 'Non classificata'

def safe_categorize_capital(classe):
    """Categorizza capitale gestendo missing values"""
    if pd.isna(classe) or classe == 'nan' or classe == '':
        return 'Dati non disponibili'
    try:
        classe_int = int(classe)
        if classe_int <= 3:
            return 'Capitale minimo (≤10k)'
        elif classe_int <= 6:
            return 'Capitale basso (10k-250k)'
        elif classe_int <= 9:
            return 'Capitale medio (250k-2.5M)'
        else:
            return 'Capitale alto (>2.5M)'
    except:
        return 'Non classificato'

def classify_innovation(req_count):
    """Classifica livello innovazione"""
    if req_count == 0:
        return 'Tradizionale'
    elif req_count == 1:
        return 'Innovativa base'
    elif req_count == 2:
        return 'Altamente innovativa'
    else:
        return 'Super innovativa'

def calculate_data_quality_score(df):
    """Calcola un punteggio di qualità dei dati per ogni startup"""
    score = pd.Series(0, index=df.index)
    
    # +1 per ogni campo critico disponibile
    critical_fields = [
        'classe di produzione ultimo anno (1)',
        'classe di addetti ultimo anno (2)',
        'classe di capitale (3)',
        '1° req.\n(6)',
        '2° req.\n(6)',
        '3° req.\n(6)',
        'sito internet'
    ]
    
    for field in critical_fields:
        if field in df.columns:
            score += df[field].notna().astype(int)
    
    return score

def classify_data_quality(score):
    """Classifica qualità dati"""
    if score >= 6:
        return 'Dati completi'
    elif score >= 4:
        return 'Dati buoni'
    elif score >= 2:
        return 'Dati parziali'
    else:
        return 'Dati insufficienti'

def calculate_startup_score_safe(df):
    """Calcola score startup gestendo missing values"""
    score = pd.Series(0, index=df.index)
    
    # Solo per startup con dati produzione disponibili
    prod_mask = df['classe di produzione ultimo anno (1)'].notna()
    prod_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8}
    score += df['classe di produzione ultimo anno (1)'].map(prod_map).fillna(0)
    
    # Solo per startup con dati addetti disponibili  
    emp_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6}
    score += df['classe di addetti ultimo anno (2)'].map(emp_map).fillna(0)
    
    # Bonus per innovazione e caratteristiche
    score += df['requisiti_soddisfatti'] * 2
    score += df.get('is_tech_energy', pd.Series(0, index=df.index)).astype(int)
    score += df.get('is_social_impact', pd.Series(0, index=df.index)).astype(int)
    score += df.get('has_website', pd.Series(0, index=df.index)).astype(int)
    
    return score

def classify_tier(score):
    """Classifica tier startup"""
    if score >= 12:
        return 'Tier 1 (Eccellente)'
    elif score >= 8:
        return 'Tier 2 (Ottima)'
    elif score >= 5:
        return 'Tier 3 (Buona)'
    else:
        return 'Tier 4 (Base)'

def generate_summary_report(df):
    """Genera report riassuntivo"""
    
    print("\n" + "="*60)
    print("📊 REPORT FINALE - STARTUP 2020-2024")
    print("="*60)
    
    print(f"📈 TOTALE STARTUP ANALIZZATE: {len(df):,}")
    print(f"📅 PERIODO: 2020-2024")
    
    # Distribuzione annuale
    print("\n📅 DISTRIBUZIONE ANNUALE:")
    year_dist = df['data iscrizione alla sezione delle startup'].dt.year.value_counts().sort_index()
    for year, count in year_dist.items():
        percentage = (count / len(df)) * 100
        print(f"  {year}: {count:,} startup ({percentage:.1f}%)")
    
    # Qualità dei dati
    print("\n📊 QUALITÀ DEI DATI:")
    quality_dist = df['data_quality_tier'].value_counts()
    for tier, count in quality_dist.items():
        percentage = (count / len(df)) * 100
        print(f"  {tier}: {count:,} startup ({percentage:.1f}%)")
    
    # Startup con dati business completi
    complete_business_data = df[
        df['produzione_descrizione'].notna() & 
        df['addetti_descrizione'].notna() & 
        df['capitale_descrizione'].notna()
    ]
    
    print(f"\n✅ STARTUP CON DATI BUSINESS COMPLETI: {len(complete_business_data):,}/{len(df):,} ({(len(complete_business_data)/len(df)*100):.1f}%)")
    
    # Distribuzione per dimensione (solo dati completi)
    if len(complete_business_data) > 0:
        print("\n🏢 DISTRIBUZIONE PER CATEGORIA (dati completi):")
        for col in ['produzione_categoria', 'addetti_categoria', 'capitale_categoria']:
            if col in complete_business_data.columns:
                print(f"\n  {col.replace('_', ' ').title()}:")
                dist = complete_business_data[col].value_counts()
                for cat, count in dist.items():
                    percentage = (count / len(complete_business_data)) * 100
                    print(f"    {cat}: {count:,} ({percentage:.1f}%)")
    
    return df

# ESECUZIONE PRINCIPALE
if __name__ == "__main__":
    # Carica e analizza dati
    df = load_and_analyze_data()
    
    if df is not None:
        # Arricchisci con gestione missing values
        df_enriched = enrich_with_legend_safe(df)
        
        # Genera report
        df_final = generate_summary_report(df_enriched)
        
        # Salva dataset pulito
        output_file = 'startup_2020_2024_CLEAN.csv'
        df_final.to_csv(output_file, index=False)
        print(f"\n💾 Dataset salvato: {output_file}")
        print("✅ ANALISI COMPLETATA CON SUCCESSO!")
    else:
        print("❌ Errore nel caricamento dati")

✅ Dataset caricato dal file Excel
Dataset iniziale: 12,320 startup
Dopo filtro 2020-2024: 9,950 startup

 ANALISI VALORI MANCANTI:
----------------------------------------
Produzione: 2,847/9,950 (28.6%)
Addetti: 5,858/9,950 (58.9%)
Capitale: 46/9,950 (0.5%)
1° Requisito: 3,700/9,950 (37.2%)
2° Requisito: 7,736/9,950 (77.7%)
3° Requisito: 8,054/9,950 (80.9%)
Vocazione Sociale: 9,770/9,950 (98.2%)
Sito Web: 1,823/9,950 (18.3%)

 PULIZIA DATI:
------------------------------
✅ Pulizia completata

🚀 ARRICCHIMENTO CON GESTIONE MISSING VALUES:
--------------------------------------------------
✅ Produzione: 7,103 mappate
✅ Addetti: 4,092 mappate
✅ Capitale: 9,904 mappate
✅ Età calcolata per 9,950 startup
✅ Arricchimento completato con gestione missing values!

📊 REPORT FINALE - STARTUP 2020-2024
📈 TOTALE STARTUP ANALIZZATE: 9,950
📅 PERIODO: 2020-2024

📅 DISTRIBUZIONE ANNUALE:
  2020: 1,577 startup (15.8%)
  2021: 2,188 startup (22.0%)
  2022: 1,774 startup (17.8%)
  2023: 1,995 startup (20.1